In [ ]:
import openai

from langsmith import Client
from qdrant_client import QdrantClient

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

### Download an example reference data points from LangGraph

In [4]:
client = Client()

In [40]:
dataset = client.read_dataset(
    dataset_name = "rag-evaluation-dataset-v3"
)

In [41]:
dataset

Dataset(name='rag-evaluation-dataset-v3', description='Evaluation dataset for RAG application', data_type=<DataType.kv: 'kv'>, id=UUID('b9bb66b2-c479-4584-8177-f1b746173f48'), created_at=datetime.datetime(2026, 1, 25, 20, 55, 15, 804082, tzinfo=TzInfo(0)), modified_at=datetime.datetime(2026, 1, 25, 20, 55, 15, 804082, tzinfo=TzInfo(0)), example_count=58, session_count=0, last_session_start_time=None, inputs_schema=None, outputs_schema=None, transformations=None, metadata={'runtime': {'sdk': 'langsmith-py', 'library': 'langsmith', 'runtime': 'python', 'platform': 'macOS-15.6.1-arm64-arm-64bit', 'sdk_version': '0.6.4', 'runtime_version': '3.12.2', 'langchain_version': None, 'py_implementation': 'CPython', 'langchain_core_version': None}})

In [42]:
list(client.list_examples(dataset_id=dataset.id, limit=10))[0].outputs

{'ground_truth': 'Yes — the LopBast 100" manual screen supports wall and ceiling installation and is compatible with 4K video. If you use a Fire TV Cube or other 4K source (which may require a downstream HDMI connection), the screen will display 4K input as it supports 4K/Ultra HD projection.',
 'reference_context_ids': ['B09Y5P41L8', 'B0B64X77P7'],
 'reference_description': ["LopBast Screen 100INCH Manual Pull Down White Projector Screen 16:9 1.2 Gain Retractable Auto-Locking 4K 8K 3D Ultra HD for Home Theater Movie Office Game Projection Screen with Slow Retract Mechanism 【High Quality Material】 Metal housing design, can effectively prevent damage and deformation during transportation. The projector screen uses a composite fabric of multiple layers, make the surface smoother and flatter, fully black backed to eliminate light penetration, has excellent projection quality. 【Manual Self-Locking Device】 Use manual self-locking design, can pull down to lock the screen to the specified pos

In [43]:
list(client.list_examples(dataset_id=dataset.id, limit=10))[0].inputs

{'question': 'Can I mount the 100" LopBast projector screen to the ceiling and will it support 4K input from a Fire TV Cube?'}

In [44]:
reference_input = list(client.list_examples(dataset_id=dataset.id, limit=10))[0].inputs
reference_output = list(client.list_examples(dataset_id=dataset.id, limit=10))[0].outputs

### RAG Pipeline

In [ ]:
def get_embeddings(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model
    )

    return response.data[0].embedding

def retrieve_data(query, qdrant_client, k=5):
    query_embedding = get_embeddings(query)
    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k
    )

    retrieved_context_ids = []
    retrieved_context= []
    similiarity_scores = []
    retrieved_context_ratings = []

    for result in results.points:
        retrieved_context_ids.append(result.payload['parent_asin'])
        retrieved_context.append(result.payload['description'])
        similiarity_scores.append(result.score)
        retrieved_context_ratings.append(result.payload['average_rating'])

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "similiarity_scores": similiarity_scores,
        "retrieved_context_ratings": retrieved_context_ratings
    }

def process_context(context):

    formatted_context = ""

    for id, chunk, rating in zip(context['retrieved_context_ids'], context['retrieved_context'], context['retrieved_context_ratings']):
        formatted_context += f"- ID: {id}, Rating: {rating}, Description: {chunk}\n"

    return formatted_context

def build_prompt(preprocessed_context, question):
    prompt = f"""
    You are a shopping assistent that can answer questions about the products in stock.
    
    You will be given a question and a list of context.
    
    Instructions:
    - You need to answer the questions based on the provided context only.
    - Never use word context and refer to it as a the available products.
    
    Context:
    {preprocessed_context}
    
    Question:
    {question}
    """

    return prompt

def generate_answer(prompt):

    response = openai.chat.completions.create(
        model="gpt-5-nano",
        messages=[{"role": "system", "content": prompt}],
        reasoning_effort="minimal"
    )

    return response.choices[0].message.content

def rag_pipeline(question, k=5):

    qdrant_client = QdrantClient(url="http://localhost:6333")

    retrieved_context = retrieve_data(question, qdrant_client, k)
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    answer = generate_answer(prompt)

    final_result = {
        "answer": answer,
        'question': question,
        "retrieved_context_ids": retrieved_context['retrieved_context_ids'],
        "retrieved_context": retrieved_context['retrieved_context'],
        "similiarity_scores": retrieved_context['similiarity_scores']
    }

    return final_result



In [45]:
rag_pipeline("can I get some charger?", k=5)

{'answer': 'Yes. Here are charger options in the available products:\n\n- B096BND451: iPhone Charger Lightning Cable 3Pack (MFi certified) — includes multiple USB cables for iPhone charging.\n- B0BKJGRZGP: 140W USB-C Charger Power Adapter for MacBook Pro / other devices — high-widelity PD 3.1 charger (no cable included).\n- B0BQXLF477: BLACKSYNCZE USB-C to Lightning Cable 2Pack (6FT) — USB-C to Lightning cables for fast charging iPhone.\n\nIf you tell me what device you’re charging (iPhone, MacBook, etc.) and whether you need a charger with an included cable, I can suggest the best fit.',
 'question': 'can I get some charger?',
 'retrieved_context_ids': ['B096BND451',
  'B0BKJGRZGP',
  'B0BQXLF477',
  'B09V24G4F8',
  'B0C3ZQ7KHJ'],
 'retrieved_context': ['iPhone Charger Lightning Cable 3Pack Quick Charger Rapid Cord Apple MFi Certified Compatible iPhone 11 Pro X XR XS MAX 8 Plus 7 6s 5s 5c Air Mini iPod ',
  "140W USB C Charger Power Adapter for MacBook Pro 16 inch 2023 2021, MacBook P

### RAGAS metrics

In [20]:
from ragas.dataset_schema import SingleTurnSample
from ragas.metrics import IDBasedContextPrecision, IDBasedContextRecall, Faithfulness, ResponseRelevancy

/var/folders/p6/st6bhdtx3sn308hvmtx5v1280000gn/T/ipykernel_49584/3756680326.py:2: DeprecationWarning: Importing IDBasedContextPrecision from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import IDBasedContextPrecision
  from ragas.metrics import IDBasedContextPrecision, IDBasedContextRecall, Faithfulness, ResponseRelevancy
/var/folders/p6/st6bhdtx3sn308hvmtx5v1280000gn/T/ipykernel_49584/3756680326.py:2: DeprecationWarning: Importing IDBasedContextRecall from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import IDBasedContextRecall
  from ragas.metrics import IDBasedContextPrecision, IDBasedContextRecall, Faithfulness, ResponseRelevancy
/var/folders/p6/st6bhdtx3sn308hvmtx5v1280000gn/T/ipykernel_49584/3756680326.py:2: DeprecationWarning: Importing Faithfulness from 'ragas.metrics' is depre

In [21]:
ragas_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))
ragas_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))

/var/folders/p6/st6bhdtx3sn308hvmtx5v1280000gn/T/ipykernel_49584/2270190839.py:1: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  ragas_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))
/var/folders/p6/st6bhdtx3sn308hvmtx5v1280000gn/T/ipykernel_49584/2270190839.py:2: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  ragas_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))


In [46]:
reference_input

{'question': 'Can I mount the 100" LopBast projector screen to the ceiling and will it support 4K input from a Fire TV Cube?'}

In [47]:
reference_output

{'ground_truth': 'Yes — the LopBast 100" manual screen supports wall and ceiling installation and is compatible with 4K video. If you use a Fire TV Cube or other 4K source (which may require a downstream HDMI connection), the screen will display 4K input as it supports 4K/Ultra HD projection.',
 'reference_context_ids': ['B09Y5P41L8', 'B0B64X77P7'],
 'reference_description': ["LopBast Screen 100INCH Manual Pull Down White Projector Screen 16:9 1.2 Gain Retractable Auto-Locking 4K 8K 3D Ultra HD for Home Theater Movie Office Game Projection Screen with Slow Retract Mechanism 【High Quality Material】 Metal housing design, can effectively prevent damage and deformation during transportation. The projector screen uses a composite fabric of multiple layers, make the surface smoother and flatter, fully black backed to eliminate light penetration, has excellent projection quality. 【Manual Self-Locking Device】 Use manual self-locking design, can pull down to lock the screen to the specified pos

In [48]:
result = rag_pipeline(reference_input["question"])

In [58]:
result

{'answer': 'Yes, you can install the LopBast 100INCH projector screen on a wall or ceiling. It supports 1080p, 4K and Full HD images, so it will accept 4K input from a Fire TV Cube.',
 'question': 'Can I mount the 100" LopBast projector screen to the ceiling and will it support 4K input from a Fire TV Cube?',
 'retrieved_context_ids': ['B09Y5P41L8',
  'B0B64X77P7',
  'B09PV64F8J',
  'B0CBQHRTXF',
  'B0BQGWMZ53'],
 'retrieved_context': ["LopBast Screen 100INCH Manual Pull Down White Projector Screen 16:9 1.2 Gain Retractable Auto-Locking 4K 8K 3D Ultra HD for Home Theater Movie Office Game Projection Screen with Slow Retract Mechanism 【High Quality Material】 Metal housing design, can effectively prevent damage and deformation during transportation. The projector screen uses a composite fabric of multiple layers, make the surface smoother and flatter, fully black backed to eliminate light penetration, has excellent projection quality. 【Manual Self-Locking Device】 Use manual self-locking 

In [50]:
async def ragas_faithfulness(run, example):
    
    sample = SingleTurnSample(
        user_input=run["question"],
        response=run["answer"],
        retrieved_contexts=run["retrieved_context"]
    )
    
    scorer = Faithfulness(llm=ragas_llm)

    return await scorer.single_turn_ascore(sample)

In [51]:
await ragas_faithfulness(result, "")

0.8333333333333334

In [52]:
async def ragas_response_relevancy(run, example):

    sample = SingleTurnSample(
        user_input=run["question"],
        response=run["answer"],
        retrieved_contexts=run["retrieved_context"]
    )

    scorer = ResponseRelevancy(llm=ragas_llm, embeddings=ragas_embeddings)

    return await scorer.single_turn_ascore(sample)

In [53]:
await ragas_response_relevancy(result, "")

np.float64(0.9312256568392329)

In [54]:
async def ragas_context_precision_id_based(run, example):

    sample = SingleTurnSample(
        retrieved_context_ids=run["retrieved_context_ids"],
        reference_context_ids=example["reference_context_ids"]
    )

    scorer = IDBasedContextPrecision()

    return await scorer.single_turn_ascore(sample)

In [55]:
await ragas_context_precision_id_based(result, reference_output)

0.4

In [56]:
async def ragas_context_recall_id_based(run, example):

    sample = SingleTurnSample(
        retrieved_context_ids=run["retrieved_context_ids"],
        reference_context_ids=example["reference_context_ids"]
    )

    scorer = IDBasedContextRecall()

    return await scorer.single_turn_ascore(sample)

In [57]:
await ragas_context_recall_id_based(result, reference_output)

1.0